# <center>Analysis of DART impact simulations</center>

## 0) Imports

In [23]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

import os
import h5py
from betafactor import betafactor

%load_ext nb_black
%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 10)
pd.set_option("display.precision", 2)

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

## 1) Beta factor:

In [41]:
project_path = "/scratch/rutz/DidymoonImpact"
folder_name = "binac_por"

filelist = sorted(os.listdir(f"{project_path}/data/{folder_name}/"))[::2]
files = [
    f"{project_path}/data/{folder_name}/{file}/impact_{file}.0300.h5"
    for file in filelist
]
files

['/scratch/rutz/DidymoonImpact/data/binac_por/por0_str1e3_ang0/impact_por0_str1e3_ang0.0300.h5',
 '/scratch/rutz/DidymoonImpact/data/binac_por/por17_str1e3_ang0/impact_por17_str1e3_ang0.0300.h5',
 '/scratch/rutz/DidymoonImpact/data/binac_por/por33_str1e3_ang0/impact_por33_str1e3_ang0.0300.h5',
 '/scratch/rutz/DidymoonImpact/data/binac_por/por5_str1e3_ang0/impact_por5_str1e3_ang0.0300.h5']

<IPython.core.display.Javascript object>

In [46]:
ejecta_thresh = 5
betas = {}
for i, file in enumerate(files):
    print(filelist[i])
    betas[file] = betafactor(file, ejecta_thresh)

por0_str1e3_ang0
Number of particles classified as ejecta: 206
Betafactor: 1.57
por17_str1e3_ang0
Number of particles classified as ejecta: 57
Betafactor: 1.20
por33_str1e3_ang0
Number of particles classified as ejecta: 76
Betafactor: 1.20
por5_str1e3_ang0
Number of particles classified as ejecta: 124
Betafactor: 1.21


<IPython.core.display.Javascript object>

## 2) Crater size: